In [1]:
from dataclasses import dataclass
from s7.graph import Graph
g = Graph()
g.clear()

g.append(('PhysicalQuantity', 'is-a', 'Quantity'))
g.append(('Salinity', 'is-a', 'PhysicalQuantity'))

g.append(('cf:sea_water_cox_salinity', 'is-a', 'Salinity'))
g.append(('cf:sea_water_knudsen_salinity', 'is-a', 'Salinity'))
g.append(('cf:sea_water_absolute_salinity', 'is-a', 'Salinity'))
g.append(('cf:sea_water_practical_salinity', 'is-a', 'Salinity'))
g.append(('cf:latitude', 'is-a', 'PhysicalQuantity'))
g.append(('cf:longitude', 'is-a', 'PhysicalQuantity'))

knudsen_to_practical_salinity = lambda s_k : (s_k - 0.03) * (1.80655 / 1.805)
g.append(('fn:knudsen_to_practical_salinity', 'is-a', 'fno:Function'))
g.append(('fn:knudsen_to_practical_salinity', 'fn:function', knudsen_to_practical_salinity))
g.append(('fn:knudsen_to_practical_salinity', 'fno:expects', 'cf:sea_water_knudsen_salinity'))
g.append(('fn:knudsen_to_practical_salinity', 'fno:returns', 'cf:sea_water_practical_salinity'))

@dataclass
class ClimateForecastData:
    """ Class containing CF data """
    lat: float = 0
    lon: float = 0
    salinity: float = 0

In [2]:
cf = ClimateForecastData (lat = 10,
                          lon = 20,
                          salinity = 0.21)
#g.append((lambda : cf.lat, 'maps-to', 'cf:latitude'))
#g.append((lambda : cf.lon, 'maps-to', 'cf:longitude'))
#g.append((lambda : cf.salinity, 'maps-to', 'cf:sea_water_knudsen_salinity'))

In [3]:
# Consumer side

consumer = ClimateForecastData()
#g.append((lambda : consumer.salinity, 'maps-to', 'cf:sea_water_practical_salinity'))

In [4]:
def path(origin, destination, link=None, visited=[]):
    if not link:
        link = [origin]    
    visited.append(origin)
    for _, r, o in g.match(origin, None, None):
        if not o in visited:
            if o == destination:                
                return link
            else:
                link.append(o)            
                return path(o, destination, visited)
    
link = path('consumer:cas', 'entity')
print (link)

entityid='<http://onto-ns/entity/1/ClimateForecastData#2c1d66e2>'
salinity_property='<http://onto-ns/entity/1/salinity#c2c0ef48>'
g.append((entityid, 'instance', cf))
g.append((entityid, 'is-a', 'Entity'))
g.append((entityid, 'has-property', salinity_property))
g.append((salinity_property, 'maps-to', 'cf:sea_water_knudsen_salinity'))
g.append(('cf:sea_water_knudsen_salinity', 'maps-from', salinity_property))




None


In [5]:
print("digraph G {")
for s,p,o in g.match():
    print (f'\t"{s}" -> "{o}" [label="{p}"];')
print("}")

digraph G {
	"PhysicalQuantity" -> "Quantity" [label="is-a"];
	"Salinity" -> "PhysicalQuantity" [label="is-a"];
	"cf:sea_water_cox_salinity" -> "Salinity" [label="is-a"];
	"cf:sea_water_knudsen_salinity" -> "Salinity" [label="is-a"];
	"cf:sea_water_absolute_salinity" -> "Salinity" [label="is-a"];
	"cf:sea_water_practical_salinity" -> "Salinity" [label="is-a"];
	"cf:latitude" -> "PhysicalQuantity" [label="is-a"];
	"cf:longitude" -> "PhysicalQuantity" [label="is-a"];
	"fn:knudsen_to_practical_salinity" -> "fno:Function" [label="is-a"];
	"fn:knudsen_to_practical_salinity" -> "<function <lambda> at 0x7fd9046b8a60>" [label="fn:function"];
	"fn:knudsen_to_practical_salinity" -> "cf:sea_water_knudsen_salinity" [label="fno:expects"];
	"fn:knudsen_to_practical_salinity" -> "cf:sea_water_practical_salinity" [label="fno:returns"];
	"<http://onto-ns/entity/1/ClimateForecastData#2c1d66e2>" -> "ClimateForecastData(lat=10, lon=20, salinity=0.21)" [label="instance"];
	"<http://onto-ns/entity/1/Climate

In [25]:
from functools import partial

def convert(value: float, factor: float, offset: float) -> float:
    return (value+offset) * factor

value = 20.0
factor = 1.80655/1.805
offset = -0.03
output = None

def set_output(v):
    global output
    output = v

class Function:
    Parameter = 'http://onto.onto-ns.com/function/1.0.0#Parameter'
    Function = 'http://onto.onto-ns.com/function/1.0.0#Function'
    Execution = 'http://onto.onto-ns.com/function/1.0.0#Execution'
    Output = 'http://onto.onto-ns.com/function/1.0.0#Output' 
    name = 'http://onto.onto-ns.com/function/1.0.0#name'
    required = 'http://onto.onto-ns.com/function/1.0.0#required'
    type = 'http://onto.onto-ns.com/function/1.0.0#type'
    expects = 'http://onto.onto-ns.com/function/1.0.0#expects'
    returns = 'http://onto.onto-ns.com/function/1.0.0#returns'
    fromValue = 'http://onto.onto-ns.com/function/1.0.0#fromValue'
    executes = 'http://onto.onto-ns.com/function/1.0.0#executes'

FNO = Function

f = Graph()
f.clear()
f.parse('file:///home/jovyan/work/semantic_interop_story/function.ttl')
f.append(('ex:convert', 'is-a', FNO.Function))
f.append(('ex:convert', FNO.expects, 'ex:value'))
f.append(('ex:convert', FNO.executes, convert))
f.append(('ex:value',   'is-a', 'fno:Parameter'))
f.append(('ex:value',   ':label', 'value'))
f.append(('ex:value',   FNO.fromValue, lambda : value))
f.append(('ex:factor',  'is-a', 'fno:Parameter'))
f.append(('ex:factor',  ':label', 'factor'))
f.append(('ex:factor',  FNO.fromValue, lambda : factor))
f.append(('ex:offset',  'is-a', 'fno:Parameter'))
f.append(('ex:offset',  ':label', 'offset'))
f.append(('ex:offset',  FNO.fromValue, lambda : offset))

f.append(('ex:convert', FNO.expects, 'ex:value'))
f.append(('ex:convert', FNO.expects, 'ex:factor'))
f.append(('ex:convert', FNO.expects, 'ex:offset'))
f.append(('ex:convert', FNO.returns, 'ex:outputValue'))
f.append(('ex:outputValue', 'is-a', 'fno:Output'))
f.append(('ex:outputValue', ':to-value', set_output))


def execute(lbl):
    for func,_,exe in f.match(lbl, FNO.executes):
        args = {}
        for _,_,parameter in f.match(func, FNO.expects):
            d = dict(zip([l for _,_,l in f.match(parameter, ':label')], 
                         [v() for _,_,v in f.match(parameter, FNO.fromValue)]))
            args.update(d)

        res = exe(**args)
        return exe(**args)    
    
execute('ex:convert')

19.987148753462606

In [17]:
print("digraph G {")
for s,p,o in f.match():
    print (f'\t"{s}" -> "{o}" [label="{p}"];')
print("}")

digraph G {
	"http://onto.onto-ns.com/function/1.0.0/OWLDataProperty_023e01b5_ae6b_4609_b58b_9bfd3a893161" -> "http://onto.onto-ns.com/function/1.0.0/OWLClass_e148914f_c5e1_4267_974d_e00e024828ac" [label="http://www.w3.org/2000/01/rdf-schema#domain"];
	"http://onto.onto-ns.com/function/1.0.0/OWLDataProperty_4d288b08_8268_4479_9743_d4bbfd0f0e89" -> "required" [label="http://www.w3.org/2000/01/rdf-schema#label"];
	"http://onto.onto-ns.com/function/1.0.0/OWLDataProperty_e1d41029_41ba_4c07_acb9_4dcde5b96555" -> "name" [label="http://www.w3.org/2000/01/rdf-schema#label"];
	"http://onto.onto-ns.com/function/1.0.0/OWLClass_c20c41ba_6e23_4484_a7e9_8db1b7939c03" -> "http://www.w3.org/2002/07/owl#Class" [label="http://www.w3.org/1999/02/22-rdf-syntax-ns#type"];
	"http://onto.onto-ns.com/function/1.0.0/OWLDataProperty_4d288b08_8268_4479_9743_d4bbfd0f0e89" -> "http://www.w3.org/2001/XMLSchema#boolean" [label="http://www.w3.org/2000/01/rdf-schema#range"];
	"http://onto.onto-ns.com/function/1.0.0/OW